<a href="https://colab.research.google.com/github/2303a52163/AIML_2303a52163/blob/main/XAI_ASS_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#XAI ASSIGNMENT-4

2303A52183

BATCH:34

**Assignment Number: 04

Q. No. Question Expected Time
to complete
1 Wine Quality (Multiclass Classification)


 Objectives:

 Apply Permutation Importance to assess the role of chemical features.
 Use SHAP to explain wine class predictions globally and locally.
 Use LIME to interpret why specific wines are classified as type X or Y.
 Compare method agreement.

 Assignment Details:

 Goal: Identify chemical properties that distinguish different wines.
 Data: sklearn.datasets.load_wine()
 Model: LogisticRegression (multi-class, standardized)
**

In [ ]:
!pip install lime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_wine
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.inspection import permutation_importance
import shap
import lime
import lime.lime_tabular

wine = load_wine()
X = pd.DataFrame(wine.data, columns=wine.feature_names)
y = wine.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = LogisticRegression(max_iter=500, multi_class='multinomial', solver='lbfgs', random_state=42)
model.fit(X_train_scaled, y_train)
print("Model Accuracy on Test Set:", model.score(X_test_scaled, y_test))

perm_importance = permutation_importance(model, X_test_scaled, y_test, n_repeats=10, random_state=42)
sorted_idx = perm_importance.importances_mean.argsort()
plt.figure(figsize=(10,6))
plt.barh(np.array(wine.feature_names)[sorted_idx], perm_importance.importances_mean[sorted_idx])
plt.xlabel("Permutation Importance")
plt.title("Permutation Importance (Wine Features)")
plt.show()

explainer = shap.LinearExplainer(model, X_train_scaled, feature_names=wine.feature_names)
shap_values = explainer(X_test_scaled)
shap.summary_plot(shap_values, X_test, feature_names=wine.feature_names)

sample_idx = 0
shap.initjs()
shap.force_plot(
    explainer.expected_value[0],
    shap_values[sample_idx].values[:, 0],
    X_test.iloc[sample_idx, :],
    feature_names=wine.feature_names
)

lime_explainer = lime.lime_tabular.LimeTabularExplainer(
    training_data=np.array(X_train_scaled),
    feature_names=wine.feature_names,
    class_names=wine.target_names,
    mode='classification'
)

for i in [0, 1]:
    exp = lime_explainer.explain_instance(X_test_scaled[i], model.predict_proba, num_features=5)
    print(f"\nLIME explanation for sample {i} (True Class: {wine.target_names[y_test[i]]})")
    exp.show_in_notebook(show_table=True)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 3.4 MB/s eta 0:00:00
